<a href="https://colab.research.google.com/github/sonnabenny/Association_Rule_Mining/blob/main/Association_Rule_Mining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas mlxtend openpyxl

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Online Retail.xlsx")
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
df = df[['InvoiceNo','Description','Country','Quantity']]

In [ ]:
df.isnull().sum()

,0
InvoiceNo,0
Description,1454
Country,0
Quantity,0


In [ ]:
df.shape

(541909, 4)

In [ ]:
# Remove leading/trailing spaces from Description
df['Description'] = df['Description'].str.strip()

# Drop missing InvoiceNo
df.dropna(subset=['InvoiceNo'], inplace=True)

# Remove transactions with 'C' (returns)
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Remove non-product items like POSTAGE, MANUAL, etc.
df = df[~df['Description'].str.contains('POSTAGE|MANUAL', case=False, na=False)]

print("Data after cleaning:", df.shape)


Data after cleaning: (530459, 4)


In [ ]:
df_fr = df[df['Country'] == "France"]

In [ ]:
df_fr.head()

,InvoiceNo,Description,Country,Quantity
26,536370,ALARM CLOCK BAKELIKE PINK,France,24
27,536370,ALARM CLOCK BAKELIKE RED,France,24
28,536370,ALARM CLOCK BAKELIKE GREEN,France,12
29,536370,PANDA AND BUNNIES STICKER SHEET,France,12
30,536370,STARS GIFT TAPE,France,24


In [ ]:
df_fr.shape

(8102, 4)

In [ ]:
#Transform Data into Transactional Format
basket = (df_fr
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().fillna(0))

# Convert to 1/0
basket_bool = basket.applymap(lambda x: 1 if x > 0 else 0)
basket_bool.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Apriori
from mlxtend.frequent_patterns import apriori
import warnings
warnings.filterwarnings("ignore")

freq_ap = apriori(basket_bool, min_support=0.01, use_colnames=True)

print("Frequent itemsets found:", freq_ap.shape)
freq_ap.head()


Frequent itemsets found: (29860, 2)


,support,itemsets
0,0.031332,(10 COLOUR SPACEBOY PEN)
1,0.015666,(12 COLOURED PARTY BALLOONS)
2,0.015666,(12 PENCIL SMALL TUBE WOODLAND)
3,0.018277,(12 PENCILS SMALL TUBE RED RETROSPOT)
4,0.018277,(12 PENCILS SMALL TUBE SKULL)


In [ ]:
from mlxtend.frequent_patterns import fpgrowth
import warnings
warnings.filterwarnings("ignore")

freq_fp = fpgrowth(basket_bool, min_support=0.01, use_colnames=True)

print("Frequent itemsets found by FP-Growth:", freq_fp.shape)
freq_fp.head()


Frequent itemsets found by FP-Growth: (29860, 2)


,support,itemsets
0,0.185379,(RED TOADSTOOL LED NIGHT LIGHT)
1,0.161880,(ROUND SNACK BOXES SET OF4 WOODLAND)
2,0.127937,(SPACEBOY LUNCH BOX)
3,0.107050,(MINI PAINT SET VINTAGE)
4,0.104439,(ALARM CLOCK BAKELIKE PINK)


In [ ]:
from mlxtend.frequent_patterns import association_rules

# Association rules from Apriori results, filtering by lift > 1
rules = association_rules(freq_fp, metric="lift", min_threshold=1)
print(rules[['antecedents','consequents', 'support', 'confidence', 'lift']])

                                   antecedents  \
0              (RED TOADSTOOL LED NIGHT LIGHT)   
1                         (RABBIT NIGHT LIGHT)   
2              (RED TOADSTOOL LED NIGHT LIGHT)   
3         (ROUND SNACK BOXES SET OF4 WOODLAND)   
4         (ROUND SNACK BOXES SET OF4 WOODLAND)   
...                                        ...   
1139923                   (RABBIT NIGHT LIGHT)   
1139924   (WOODEN STAR CHRISTMAS SCANDINAVIAN)   
1139925  (WOODEN HEART CHRISTMAS SCANDINAVIAN)   
1139926          (SMALL RED BABUSHKA NOTEBOOK)   
1139927       (SMALL YELLOW BABUSHKA NOTEBOOK)   

                                   consequents   support  confidence  \
0                         (RABBIT NIGHT LIGHT)  0.054830    0.295775   
1              (RED TOADSTOOL LED NIGHT LIGHT)  0.054830    0.283784   
2         (ROUND SNACK BOXES SET OF4 WOODLAND)  0.036554    0.197183   
3              (RED TOADSTOOL LED NIGHT LIGHT)  0.036554    0.225806   
4           (PLASTERS IN TIN WOODLAND A

In [ ]:
print("\nGenerating association rules (metric='lift', min_threshold=1)...")
# We will use a common threshold for confidence as well, e.g., 0.5
rules = association_rules(freq_fp, metric="lift", min_threshold=1)

# Sort the rules by 'lift' to see the strongest associations first
rules_sorted = rules.sort_values('lift', ascending=False)

print("\n--- Top Association Rules (Sorted by Lift) ---")
# Displaying the columns you requested
print(rules_sorted[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10).to_string(index=False))

print("\n--- Interpretation Example (first rule) ---")
if not rules_sorted.empty:
    first_rule = rules_sorted.iloc[0]
    print(f"Rule: IF a customer buys {list(first_rule['antecedents'])},")
    print(f"THEN they are {first_rule['lift']:.2f} times MORE LIKELY")
    print(f"to also buy {list(first_rule['consequents'])}.")
    print(f"This rule has a confidence of {first_rule['confidence']:.0%}.")
else:
    print("No rules found with the current thresholds.")


Generating association rules (metric='lift', min_threshold=1)...

--- Top Association Rules (Sorted by Lift) ---
                                                                                                         antecedents                                                                                                                                    consequents  support  confidence  lift
                                    (LUNCH BAG APPLE DESIGN, SKULL LUNCH BOX WITH CUTLERY, ALARM CLOCK BAKELIKE RED)       (RED RETROSPOT MINI CASES, JUMBO BAG APPLES, CHILDRENS CUTLERY DOLLY GIRL, ALARM CLOCK BAKELIKE GREEN, LUNCH BOX WITH CUTLERY RETROSPOT) 0.010444         1.0 95.75
                          (CHILDRENS CUTLERY DOLLY GIRL, LUNCH BOX WITH CUTLERY RETROSPOT, ALARM CLOCK BAKELIKE RED)                 (RED RETROSPOT MINI CASES, JUMBO BAG APPLES, SKULL LUNCH BOX WITH CUTLERY, ALARM CLOCK BAKELIKE GREEN, LUNCH BAG APPLE DESIGN) 0.010444         1.0 95.75
                         